In [40]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr 
import json

In [41]:
load_dotenv(override=True)

GROQ_BASE_URL = "https://api.groq.com/openai/v1"
api_key = os.getenv('GROQ_API_KEY')

if api_key:
    print(f"API key starting with {api_key[:6]} is found and Good to Go!")
else:
    print("There was an issue fetching API key")

groq = OpenAI(base_url=GROQ_BASE_URL, api_key=api_key)

MODEL = "llama-3.1-8b-instant"

API key starting with gsk_sz is found and Good to Go!


In [ ]:
system_message = """
You are a helpful assistant for an airline called FlightAI.

When a user asks about ticket prices, you MUST call the tool
`get_ticket_price` with the destination city.

Do NOT guess prices.
Use the tool whenever price information is needed.

Give short, courteous answers, no more than 1 sentence.
If you don't know the answer, say so.
"""

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = groq.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

gr.ChatInterface(fn=chat).launch(inbrowser=True)

In [42]:
ticket_prices = {"tokyo": "$200", "berlin": "$250", "denver": "$320", "nairobi": "$350"}

def get_ticket_price(destination_city):
    print(f"Tool called for {destination_city}.")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of ticket {destination_city} is {price}"

In [58]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

set_price_function = {
    "name": "set_ticket_price",
    "description": "Set or update the ticket price for a destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city whose ticket price needs to be set"
            },
            "price": {
                "type": "string",
                "description": "Ticket price in USD"
            }
        },
        "required": ["city", "price"],
        "additionalProperties": False
    }
}

In [44]:
tools = [{"type": "function", "function": price_function}, {"type": "function", "function": set_price_function}]

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = groq.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = groq.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [ ]:
gr.ChatInterface(fn=chat).launch(inbrowser=True)

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = groq.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_call(message)
        messages.append(message)
        messages.extend(responses)
        response = groq.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [ ]:
gr.ChatInterface(fn=chat).launch(inbrowser=True)

In [45]:
import sqlite3

In [46]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)")
    conn.commit()

In [47]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT price FROM prices WHERE city = ?", (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [54]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()
    return f"Ticket price for {city.title()} set to ${price}."

In [59]:
ticket_prices = {"london":"799", "paris": "899", "tokyo": "1420", "sydney": "2999"}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

In [ ]:
gr.ChatInterface(fn=chat).launch(inbrowser=True)

## *AI AIRLINE ASSISTANT TO SET AND GET PRICES*

In [56]:
system_message = """ 
You are a helpful assistant for an airline called FlightAI.

When a user asks about ticket prices, you MUST call the tool
`get_ticket_price`.

When a user asks to set or update a ticket price, you MUST call
the tool `set_ticket_price`.

Do NOT explain tool usage.
Do NOT mention functions, tools, or internal actions.
Respond ONLY with the final answer for the user.

Give short, courteous answers, no more than 1 sentence.
If you don't know the answer, say so.
"""

In [51]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = groq.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_call(message)
        messages.append(message)
        messages.extend(responses)
        response = groq.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [52]:
def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)

        if tool_call.function.name == "get_ticket_price":
            city = arguments.get('destination_city')
            result = get_ticket_price(city)

        elif tool_call.function.name == "set_ticket_price":
            city = arguments.get("city")
            price = arguments.get("price")
            result = set_ticket_price(city, price)
        
        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id
        })

    return responses

In [ ]:
gr.ChatInterface(fn=chat).launch()